# Ejercicio de programación  8:
# Detección de Anomalías y Sistemas de Recomendación


## Introducción

En este ejercicio usted implementará el algoritmo de detección de anomalías
y las applicará para detectar fallas en servidores en un red. En la segunda parte
usará el filtro colaborativo para construir un sistema de recomendación de películas.
Antes de comenzar le recomendamos revisar las notas de clase.

Toda la información requerida para resolver esta tarea está en este cuadernillo. Además
todo el código que implementará también está en este cuadernillo. 

Antes de comenzar necesitará importar todas las librerías requeridas para el ejercicio. Usted podría requerir [`numpy`](http://www.numpy.org/)  para las operaciones entre arreglos y matrices,  [`matplotlib`](https://matplotlib.org/) 
para graficar, y [`scipy`](https://docs.scipy.org/doc/scipy/reference/)  para cómputos numéricos.

In [ ]:
# para manipulación de caminos y archivos
import os

# para cálculos numéricos
import numpy as np

# para graficar
from matplotlib import pyplot
import matplotlib as mpl

# librería de optimización
from scipy import optimize

# para cargar el archivo en formato MATLAB
from scipy.io import loadmat

# library written for this exercise providing additional functions for assignment submission, and others
# librería con funciones adicionales que se podrían necesitar
import utils

# grafique dentro del cuadernillo
%matplotlib inline

## 1 Detección de Anomalías 

En este ejercicio implementará el algoritmo de detección de anomalías para detectar el comportamiendo de servidores de computación. Los atributos miden el  rendimiento (mb/s) y latencia (ms) de la respuesta de cada servidor. Mientras los
servidores estan operando, usted recolecta $m=307$ muestras acerca del comportamiento, de forma que tiene unos datos
no supervisados (sin etiquetas)  $\{x^{(1)}, \dots, x^{(m)}\}$. Usted sospecha que la gran mayoría de estas muestras son
"normales" (no son anomalías), pero podrían haber algunas muestras de servidores actuando anómalamente con este conjunto de datos. 

Usted usará un modelo Gausiano para detectar anomalías en su conjunto de datos.  Inicialmente comienza con un conjunto 2D de datos que le permitirá visualizar lo que el algoritmo hace.  Usted ajusta el conjunto de datos a una distribución Gausiana
y encuentra los valores que tienen probabilidad baja y considerados como anómalos. Luego de esto usted aplica el algoritmo de detección de anomalías a un conjunto de más dimensiones. 

Comenzamos el ejercicio usando un conjunto de datos pequeño y fácil de visualizar. Nuestro ejemplo consiste en 2 estadísticas
de servidores de red a lo largo de varias máquinas: la latencia y el rendimiento


In [ ]:
#  cargamos los datos
data = loadmat(os.path.join('Data', 'ex8data1.mat'))
X, Xval, yval = data['X'], data['Xval'], data['yval'][:, 0]

#  visualizamos este ejemplo
pyplot.plot(X[:, 0], X[:, 1], 'bx', mew=2, mec='k', ms=6)
pyplot.axis([0, 30, 0, 30])
pyplot.xlabel('Latency (ms)')
pyplot.ylabel('Throughput (mb/s)')
pass

### 1.1 Distribución Gausiana

Para ejecutar la detección de anomalías usted necesita primero ajustar un modelo a la distribución de los datos. 
Dado un conjunto de datos $\{x^{(1)}, \dots, x^{(m)} \}$ (donde  $x^{(i)} \in \mathbb{R}^n$ ), se debe estimar
la distribución Gausiana para cada uno de los atributos $x_i$. Para cada atributo $i=1, \dots n$, necesita calcular
los parámetros $\mu_i$ y $\sigma_i^2$ que se ajustan a los datos en la 
dimensión $i$, $\{ x_i^{(1)}, \dots, x_i^{(m)} \}$ (la dimensión $i$
de cada muestra).


La distribución Gausiana está data por

$$ p\left( x; \mu, \sigma^2 \right) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{\left(x-\mu\right)^2}{2\sigma^2}},$$
donde $\mu$ es la media y  $\sigma^2$ la varianza.

<a id="section1"></a>
### 1.2 estimado de parámetros Gausianos

Usted puede estimar los parámetros $\left( \mu_i, \sigma_i^2 \right)$, 
del atributo $i$ mediante el uso de las siguientes ecuaciones:

media
$$ \mu_i = \frac{1}{m} \sum_{j=1}^m x_i^{(j)},$$

varianza

$$ \sigma_i^2 = \frac{1}{m} \sum_{j=1}^m \left( x_i^{(j)} - \mu_i \right)^2.$$

Su tarea es completar el código `estimateGaussian`.  Esta función asume
como entrada la matriz `X`  y debería sacar el vector n-dimensional 
`mu`, que guarda las medias de cada una de los $n$ atributos y 
el vector n-dimensional   `sigma2` contiene las varianzas de cada uno de los atributos.
Usted puede impelentar esto usando un bucle for sobre cada atributo y muestra (aunque una
implementación vecrtorizada podría ser más eficiente; siéntase libr de tratar una implementación
vectorizada si así lo prefiere.

<a id="estimateGaussian"></a>

In [ ]:
def estimateGaussian(X):
    """
    Esta función estima los parámetros de una distribución Gaussiana
    usando los datos suminstrados.
    
    Parámetros
    ----------
    X : arreglo
        los datos de tamaño (m x n) con m puntos n-dimensionales
    
    Retorna
    -------
    mu : arreglo
        Un vector de tamaño (n,) con las medias en cada dimensón.
    
    sigma2 : arreglo
        Un vector de tamaño  (n,) con las varianzas en cada dimensión.
    
    Instrucciones
    ------------
    Calcule la media de los datos y las varianzas.
    In particular ,mu[i] debe contener la media de
    los datos para el atributo $i$ y sigma2[i]
    debe contener la varianza para el atributo i.
    """
    # variables útiles
    m, n = X.shape

    # debe retornar estos valores correctamente
    mu = np.zeros(n)
    sigma2 = np.zeros(n)

    # ====================== Su Código Acá ======================

    
    # =============================================================
    return mu, sigma2

Una vez halla completado el código en `estimateGaussian`, la celda siguiente visualiza los contornos de
la distribución Gausiana ajustada. Debería obenter una figura como la siguiente.

![](Figures/gaussian_fit.png)

De la figura, usted puede ver las mayoría de las muestras en la región de más alta propbabilidad, mientras
que las muestras atípicas están en regiones de baja probabilidad. 

Para la visualización del ajuste Gausiano, primero estimamos los parametros de la distribución y luego
calculamos la probabilidad para cada uno de los puntos. Luego visualizamos la distribución en general donde
cada punto se localiza de acuerdo a la distribución.

In [ ]:
#  Estime mu y sigma2
mu, sigma2 = estimateGaussian(X)

#  Retorna la densidad multivariada de la normal para cada punto (fila) de X
p = utils.multivariateGaussian(X, mu, sigma2)

#  Visualizamos el ajuste
utils.visualizeFit(X,  mu, sigma2)
pyplot.xlabel('Latency (ms)')
pyplot.ylabel('Throughput (mb/s)')
pyplot.tight_layout()

<a id="section2"></a>
### 1.3 Seleccione el umbral,  $\varepsilon$

Ahora que ya tiene los parámetros Gausianos, puede investigar que muestras tienen una alta probabilidad
dada la distribución y que muestras tienen una baja probabilidad. Las muestras de baja probabilidad son 
las más factibles de ser anómalas. Una forma de determinar que muestras son anómalas es mediante la}
selección de un umbral basado en un conjunto de validación cruzada.  En esta parte del ejercicio usted
implementará un algoritmo para seleccionar el umbral $\varepsilon$ mediante el uso del $F_1$ score,
en el conjunto de validación cruzada. 

Usted debe ahora completar ls función  `selectThreshold`.  Para esto, usaremos una validación cruzada
en el conjunto  $\{ (x_{cv}^{(1)}, y_{cv}^{(1)}), \dots, (x_{cv}^{(m_{cv})}, y_{cv}^{(m_{cv})})\}$, 
donde la etiqueta $y=1$ corresponde a la muestra anómala, y $y=0$ corresponde a una muestra normal. Para
cada muestra de validación cruzada, calculamos  $p\left( x_{cv}^{(i)}\right)$. El vector de todas las probabilidades
s $p\left( x_{cv}^{(1)}\right), \dots, p\left( x_{cv}^{(m_{cv})}\right)$  se le pasa a `selectThreshold` en el 
vector `pval`.

La  función `selectThreshold` debería retornar dos valores el primero es el umbral seleccionado
$\varepsilon$. Si una muestra $x$ tiene baja probabilidad $p(x) < \varepsilon$, entonces se considera una anomalía.
El segundo es el $F_1$ score, que nos dice que tan bien estamos estimando los valores calculados con respecto a los reales, de acuerdo al umbral establecido. Para muchos umbrales $\varepsilon$, debe comparar las diferentes
$F_1$ medidas basado en el la clasificación de las muestras.

El $F_1$ score se calcula usando la precisión ($prec$) y la sensibilidad (recall, $rec$).

$$ F_1 = \frac{2 \cdot prec \cdot rec}{prec + rec}, $$

Usted calcula la precición y sensibilidad mediante las fórmulas
$$ prec = \frac{tp}{tp + fp}  $$ 

$$ rec = \frac{tp}{tp + fn} $$

donde:

- $tp$ es el número de verdaderos positivos (true positives): La etiqueta real dice que es una anomalía y
el algoritmo  correctmente la clasifica como tal.

-  $fp$ es el número de falsos positivos: la etiqueta real dice que no es una anomalía, pero el algoritmo
incorrectamente lo clasifica como anomalía.
- $fn$ es el número de falsos negativos:  la etiqueta real dice que es una anomalía, pero el algoritmo
- incorrectamente lo calsifica como normal.

En el código suminstrado `selectThreshold`, hay un ciclo que prueba muchos umbrales $\varepsilon$
y selecciona el mejor $\varepsilon$ basado en el $F_1$ score. Usted debe completar el código `selectThreshold`.
Puede implementar el cómputo de $F_1$ usando un bucle for sobre todas las muestras de validación cruzada
(para calcular los valores de $tp$, $fp$, $fn$). Debería tener un `epsilon` cercano a 8.99e-05.


<div class="alert alert-block alert-warning">
    
**Nota de implementación**  :  Con el fin de calcular $tp$, $fp$ y $fn$, podría usar una implementación 
vectorizada mejor que un bucle sobre las muestras.  Esta se puede implementar usando una prueba de igualdad
en numpy entre un vector y un escalar. Si tiene varias muestras vinarias en un vector binario n-dimensional
$v \in \{0, 1\}^n$, puede encontrar cuantas muestras en el vector son 0 usando: `np.sum(v==0)` . 
Tambien puede aplicar un operador lógico sobre los vectores binarios. Por ejemplo, sea  `cvPredictions` 
un vector binario de tamaño $x_{cv}^{(i)}$ una anomalía, y 0 si no. Usted puede, por ejemplo, calcular
el número de falsos positivos usando: `fp = np.sum((cvPredictions == 1) & (yval == 0))`.
</div>
<a id="selectThreshold"></a>

In [ ]:
def selectThreshold(yval, pval):
    """
    Encuentre el mejor umbral (epsilon) para usar en la selección de
    muestras anómalas basdo en resultados del conjunto de validación 
    cruzada y las etiquetas de referencia reales.
    
    Parámetros
    ----------
    yval : arreglo
        El arreglo de valores reales, de tamaño (m, ).
    
    pval : arreglo
        El vector precalculado de probabilidades basado en la media mu y la varianza sigma2.
        Tiene tamaño (m, ).
    
    Retorna
    -------
    bestEpsilon : arreglo
        Un vector de tamaño (n, ) correspondiente al valor del umbral.
    
    bestF1 : float
        El valor del mejor F1 score.
    
    Instrucciones
    ------------
    Calcule el F1 score escogiendo epsilon como el umbral y coloque
    el valor in F1. El código al final del ciclo compara el F1 score
    para esta escogencia de epsilon y lo asigna como el mejor epsilon
    si es mejor que el la elección acual de epsilon.
    
    Notas
    -----
    Usted puede usar las predicciones =(pval < epsilon) para obtener
    el vector de 0s y 1s en la predicción de muestras atípicas. 
    """
    bestEpsilon = 0
    bestF1 = 0
    F1 = 0
   
    for epsilon in np.linspace(1.01*min(pval), max(pval), 1000):
        # ====================== Su Código Acá =======================

        
        

        # =============================================================
        if F1 > bestF1:
            bestF1 = F1
            bestEpsilon = epsilon

    return bestEpsilon, bestF1

Una vez halla completado el código en `selectThreshold`, la próxima celdda calcula la detección de anomalías y 
las marca con  un círculo rojo.

In [ ]:
pval = utils.multivariateGaussian(Xval, mu, sigma2)

epsilon, F1 = selectThreshold(yval, pval)
print('Best epsilon found using cross-validation: %.2e' % epsilon)
print('Best F1 on Cross Validation Set:  %f' % F1)
print('   (you should see a value epsilon of about 8.99e-05)')
print('   (you should see a Best F1 value of  0.875000)')

#  Find the outliers in the training set and plot the
outliers = p < epsilon

#  Visualize the fit
utils.visualizeFit(X,  mu, sigma2)
pyplot.xlabel('Latency (ms)')
pyplot.ylabel('Throughput (mb/s)')
pyplot.tight_layout()

#  Dibuje un círculo rojo al rededor de cada muestra anómala.
pyplot.plot(X[outliers, 0], X[outliers, 1], 'ro', ms=10, mfc='None', mew=2)
pass

### 1.4 Conjunto con muchas dimensiones

La próxima celda corre el algoritmo de detección de anomalías implementado en datos mas realistas y difíciles. 
En estos datos, cada muestra tiene 11 atributos, los cuales capaturan muchas propiedades de servidores de computación, donde solo unos pocos atributos indican que punto es anómalo. El código usa su código para
estimar los parámetros Gausianos ($\mu_i$ y $\sigma_i^2$), evalúan las probabilidades para los datos de 
entrenamiento `X` y validación  `Xval`. Finalmente, usa `selectThreshold`  para encontrar el mejor
umbral $\varepsilon$. Usted debería ver un epsilon cercano a 1.38e-18, y 117 anomalías.


In [ ]:
#  Cargue los datos. Debería tener
#  variables X, Xval, yval en su ambiente
data = loadmat(os.path.join('Data', 'ex8data2.mat'))
X, Xval, yval = data['X'], data['Xval'], data['yval'][:, 0]

# Aplicar los mismos pasos para el conjunto grande
mu, sigma2 = estimateGaussian(X)

#  Entrenamiento
p = utils.multivariateGaussian(X, mu, sigma2)

#  Validación cruzada.
pval = utils.multivariateGaussian(Xval, mu, sigma2)

#  Encuentre el mejor umbral
epsilon, F1 = selectThreshold(yval, pval)

print('Best epsilon found using cross-validation: %.2e' % epsilon)
print('Best F1 on Cross Validation Set          : %f\n' % F1)
print('  (you should see a value epsilon of about 1.38e-18)')
print('   (you should see a Best F1 value of      0.615385)')
print('\n# Outliers found: %d' % np.sum(p < epsilon))

## 2 Sistemas de Recomendación

En esta parte del ejercicio, usted debe implementar un algoritmo de filtros colaborativos y aplicarlos
a un conjunto de calificación de películas ([MovieLens 100k Dataset](https://grouplens.org/datasets/movielens/) 
de GroupLens Reseach). Este conunto de datos tiene una escala de 1 a 5. Los datos tienen $n_u=943$ usuarios,
y $n_m=1682$ películas.

En las siguientes partes del ejercicio usted debe implementar la función `cofiCostFunc`  que calcula
la función objetivo y gradiente de los filtros colaborativos. Luego de implementar la función de costo y 
el gradiente, usted debe usar `scipy.optimize.minimize` para aprender los parámetros de los filtros colaborativos. 


### 2.1 Datos de calificación de películas

La próxima celda carga los datos `ex8_movies.mat`,  que generan las variables `Y` y `R`.
La matriz `Y` (de tamaño  `num_movies` $\times$ `num_users`) guarda las calificaciones $y^{(i,j)}$
(de 1 a 5).  La matriz binaria `R`  indica donde $R(i,j) = 1$ si el usuario $j$  calificó la película
$i$, y $R(i,j)=0$ en caso contrario.  El objeto de los filtros colaborativos es predecir las calificaciones
de las películas que los usuarios no han calificado. Es decir, los valores $R(i,j)=0$. Esto permite la recomendación de una película con la mejor predicción al usuario.

Para mejor entender la matriz `Y`,  la celda que sigue calcula la calificación media para la primera película
(Toy Story) e imprime este valor.

In [ ]:
# Cargamos los datos
data = loadmat(os.path.join('Data', 'ex8_movies.mat'))
Y, R = data['Y'], data['R']

# Y es una matriz  1682x943 con las calificaciones (1-5) de
# 1682 películas y  943 usuarios

# R es una matriz  1682x943 matrix, donde  R(i,j) = 1 
# si y solo si el usuario j calificó la película i

# de la matriz podemos calcular las estadísticas tales como el promedio de calificación.
print('Average rating for movie 1 (Toy Story): %f / 5' %
      np.mean(Y[0, R[0, :] == 1]))

# Podemos "visualizar" la matriz de calificaciones graficandola con imshow
pyplot.figure(figsize=(8, 8))
pyplot.imshow(Y)
pyplot.ylabel('Movies')
pyplot.xlabel('Users')
pyplot.grid(False)

En esta parte del ejercicio usted trabajará con las matrices `X` y `Theta`:

$$ \text{X} = 
\begin{bmatrix}
- \left(x^{(1)}\right)^T - \\
- \left(x^{(2)}\right)^T - \\
\vdots \\
- \left(x^{(n_m)}\right)^T - \\
\end{bmatrix}, \quad
\text{Theta} = 
\begin{bmatrix}
- \left(\theta^{(1)}\right)^T - \\
- \left(\theta^{(2)}\right)^T - \\
\vdots \\
- \left(\theta^{(n_u)}\right)^T - \\
\end{bmatrix}.
$$

La fila $i$ de  `X`  corresponde al vector de atributos $x^{(i)}$  para la película $i$ y la 
fila $j$ de `Theta` corresponde a un vector paramétro $\theta^{(j)}$ para la película $i$, y la fila
$j$ de `Theta`  corresponde a un parámetro del vector $\theta^{(j)}$, para el usuario $j$.
Ambos $x^{(i)}$ y $\theta^{(j)}$ son vectores n dimensionales. Para este ejercicio  used usará
$n=100$, y por eso $x^{(i)} \in \mathbb{R}^{100}$ y $\theta^{(j)} \in \mathbb{R}^{100}$.
Correspondientemente, `X` es una matriz $n_m \times 100$ y `Theta` es una matriz  $n_u \times 100$.


<a id="section3"></a>
### 2.2 Algoritmo de aprendizaje de filtros colaborativos

En este momento usted comienza con la implementación del algoritmo de filtros colaborativos. Inicialmente
implementará la función de costo (sin regularización).

El algoritmo de filtros colaborativos en el contexto de recomendación de películas considera un conjunto
n dimensional de parámetros $x^{(1)}, \dots, x^{(n_m)}$ y $\theta^{(1)} , \dots, \theta^{(n_u)}$, 
donde el modelo predice la calificación de la película $i$ por el usuario $j$ como $y^{(i,j)} = \left( \theta^{(j)} \right)^T x^{(i)}$. Dado un conjunto de datos de calificaciones producido por algunos usuarios y algunas
películas, usted desea aprender los vectores $x^{(1)}, \dots, x^{(n_m)}, \theta^{(1)}, \dots, \theta^{(n_u)}$
que producen el mejor ajuste (minimiza el cuadrado del error). 

Debe completar el código en  `cofiCostFunc`  para calcular la función de costo y el gradiente de los filtros
colaborativos. Note que los parámetros de la función (es decir, los valors que está tratando de entender)
son `X` y `Theta`. Para poder usar un minimizador disponible tal como la función `minimize` de la librería
`scipy`, la función de costo se postula para desarrollar los parámetros en un solo vector llamado `params`. 
Usted ha usado previamente este tipo de técnica en el ejercicio de redes neuronales.


#### 2.2.1 Filtros colaborativos con función de costo

La función de costo para filtros colaborativos (sin regularización) está dada por

$$
J(x^{(1)}, \dots, x^{(n_m)}, \theta^{(1)}, \dots,\theta^{(n_u)}) = \frac{1}{2} \sum_{(i,j):r(i,j)=1} \left( \left(\theta^{(j)}\right)^T x^{(i)} - y^{(i,j)} \right)^2
$$

Usted debería 
You should now modify the function `cofiCostFunc` to return this cost in the variable `J`. Note that you should be accumulating the cost for user $j$ and movie $i$ only if `R[i,j] = 1`.

<div class="alert alert-block alert-warning">
**Implementation Note**: We strongly encourage you to use a vectorized implementation to compute $J$, since it will later by called many times by `scipy`'s optimization package. As usual, it might be easiest to first write a non-vectorized implementation (to make sure you have the right answer), and the modify it to become a vectorized implementation (checking that the vectorization steps do not change your algorithm’s output). To come up with a vectorized implementation, the following tip might be helpful: You can use the $R$ matrix to set selected entries to 0. For example, `R * M` will do an element-wise multiplication between `M`
and `R`; since `R` only has elements with values either 0 or 1, this has the effect of setting the elements of M to 0 only when the corresponding value in R is 0. Hence, `np.sum( R * M)` is the sum of all the elements of `M` for which the corresponding element in `R` equals 1.
</div>

<a id="cofiCostFunc"></a>

In [ ]:
def cofiCostFunc(params, Y, R, num_users, num_movies,
                      num_features, lambda_=0.0):
    """
    Función de costo para filtros colaborativos
    
    Parámetros
    ----------
    params : array
        Parámetros a optimizar. Vector unidimensional
        de tamaño (nummero de películas x número de usarios, 1).
        Es la concatenación del vector de atributos X y los parámetros Theta.
    
    Y : arreglo
        Una matriz de tamaño (num_movies x num_users) de calificaciones 
        de películas de usuarios.
    
    R : arreglo
        Una matriz (num_movies x num_users) matrix, donde R[i, j] = 1 si la
        película i fue calificada por el usuario j.
    
    num_users : int
        Número total de usuarios.
    
    num_movies : int
        Número total de películas.
    
    num_features : int
        Número de atributos a aprender
    
    lambda_ : float, opcional
        Coeficiente de regularización
    
    Retorna
    -------
    J : float
        Valor de la función de costo para los parámetros datos. 
    
    grad : arreglo
        Gradiente del vector de costo como función de los parámetros.
        Tiene tamaño   (num_movies x num_users, 1).
    
    Instrucciones
    ------------
    Calcula la función de costo y su gradiente de los filtros colaborativos.
    Concretamente, debe implementar primero la función de costo (sin 
    regularización) y verificar que se ajusta a nuestros costos. Luego 
    debe implementar el gradiente correctamente. Finalmente, debe implementar
    la parte de la regularización.
    
    
    Notas
    -----
    - Los parámetros se reparten en dos matrices:
        X : matriz de tamaño (num_movies  x num_features) con los atributos
        Theta : de tamañp (num_users  x num_features) con los atributos de usuario

    - Debe asignar las siguientes variables correctamente:

        X_grad : matriz de tamaño (num_movies x num_features) con 
                 derivadas parciales con respecto a cada elemento de X
        Theta_grad : matrix de tamaño (num_users x num_features) con
                     derivadas parciales con respecto a cada elemento de Theta

    - El gradiente de retorno es la concatenación de los gradientes
      de X,  X_grad and Theta, Theta_grad.
    """
    # Depliege las matrices U y W de los parámetros
    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)

    # Debe retornar las siguientes variables correctamente
    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)

    # ====================== Su Código Acá ======================

    
    
    # =============================================================
    
    grad = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])
    return J, grad

Luego de completar la función, la celda siguiente corre la función de costo. Para ayudarle a depurar la función de costo, incluimos un conjunto de pesos previamente entrenados. Debe esperar, como salida 22.22.


In [ ]:
#  Cargue los pesos pre-entrenados (X, Theta, num_users, num_movies, num_features)
data = loadmat(os.path.join('Data', 'ex8_movieParams.mat'))
X, Theta, num_users, num_movies, num_features = data['X'],\
        data['Theta'], data['num_users'], data['num_movies'], data['num_features']

#  Reduzca los datos para que el programa corra más rápido
num_users = 4
num_movies = 5
num_features = 3

X = X[:num_movies, :num_features]
Theta = Theta[:num_users, :num_features]
Y = Y[:num_movies, 0:num_users]
R = R[:num_movies, 0:num_users]

#  Evalúe la función de costo
J, _ = cofiCostFunc(np.concatenate([X.ravel(), Theta.ravel()]),
                    Y, R, num_users, num_movies, num_features)
           
print('Cost at loaded parameters:  %.2f \n(this value should be about 22.22)' % J)

<a id="section4"></a>
#### 2.2.2 Gradiente de filtros colaborativos

En este punto debe implementar el gradiente (sin regularización). Específicamente, usted debería completar
el código `cofiCostFunc` que retorna las variables `X_grad` y `Theta_grad`.  Note que `X_grad` debe ser
una matriz del mismo tamaño que `X` y similarmente, `Theta_grad` es una matriz del mismo tamaño que
`Theta`.  Los gradientes de la función de costo están dados por:


$$ \frac{\partial J}{\partial x_k^{(i)}} = \sum_{j:r(i,j)=1} \left( \left(\theta^{(j)}\right)^T x^{(i)} - y^{(i,j)} \right) \theta_k^{(j)} $$

$$ \frac{\partial J}{\partial \theta_k^{(j)}} = \sum_{i:r(i,j)=1} \left( \left(\theta^{(j)}\right)^T x^{(i)}- y^{(i,j)} \right) x_k^{(i)} $$


Note que la función retorna el gradiente para ambos conjuntos de variables plegándolos en un solo vector.
Luego de completar el código para calcular los gradientes, la próxima celda le permite verificar el gradiente
(disponible en `utils.checkCostFunction`) mediante la verificación de la implementación de sus gradientes
(esto es similar al chequeo numérico que se usó en el ejercicio de redes neuronales). Si su implementación
es correcta, usted debería encontrar que el gradiente analítico y numérico deben estar cerca. 


<div class="alert alert-block alert-warning">
    
**Nota de Imlementación**: Puede obtener crédito total sin usar una implementación vectorizada, solo que
su código correrá mucho mas lento (un poco número de horas), de forma que recomendamos que trate de 
vectorizar su implementación. Para comenzar, usted puede implementar el gradiente con un ciclo °for°
sobre películas (para calcular  $\frac{\partial J}{\partial x^{(i)}_k}$) y un ciclo "for" sobre 
usuarios (para calcular $\frac{\partial J}{\theta_k^{(j)}}$). Cuando implemente el gradiente inicialmente,
podría comenzar con una implementación no vectorizada, mediante la implementación de un ciclo "for" que calcula
cada elemento de la suma. Luego puede tratar de vectorizar su implementación (vectorice los ciclos internos "for"),
de forma que solo termina con dos ciclos "for" (uno sobre las películas para calcular $\frac{\partial J}{\partial x_k^{(i)}}$ para cada película, y otro ciclo "for" sobre usuarios para calcular $\frac{\partial J}{\partial \theta_k^{(j)}}$ por usuario.


</div>

<div class="alert alert-block alert-warning">
    
**Sugerencia de Implementación:**  Para implementar la vectorización podría ser útil la búsqueda
una forma de calcular todas las derivadas asociadas con $x_1^{(i)} , x_2^{(i)}, \dots , x_n^{(i)}$ 
(es decir, los términos derivados asociados con el vector de atributos  $x^{(i)}$) al mismo tiempo.
Definamos las derivadas del vector de atributos de la película $i$:
    

$$ \left(X_{\text{grad}} \left(i, :\right)\right)^T = 
\begin{bmatrix}
\frac{\partial J}{\partial x_1^{(i)}} \\
\frac{\partial J}{\partial x_2^{(i)}} \\
\vdots \\
\frac{\partial J}{\partial x_n^{(i)}}
\end{bmatrix} = \quad
\sum_{j:r(i,j)=1} \left( \left( \theta^{(j)} \right)^T x^{(i)} - y^{(i,j)} \right) \theta^{(j)}
$$

Para vectorizar la expresión de arriba puede comenzar con indizar `Theta` y `Y` para seleccionar solo
los elementos de interés (es decir, `r[i, j] = 1`).  Intuitivamente, cuando considere los atributos
para la película $i$, solo se debe preocupar por usuarios que han calificado la película, esto le permite
remover todos los usuarios de Theta` y `Y`. <br/><br/>


Concretely, you can set `idx = np.where(R[i, :] == 1)[0]` to be a list of all the users that have rated movie $i$. This will allow you to create the temporary matrices `Theta_temp = Theta[idx, :]` and `Y_temp = Y[i, idx]` that index into `Theta` and `Y` to give you only the set of users which have rated the $i^{th}$ movie. This will allow you to write the derivatives as: <br>

`X_grad[i, :] = np.dot(np.dot(X[i, :], Theta_temp.T) - Y_temp, Theta_temp)`

<br><br>
Note que la versión vectorizada de arriba retorna un vector fila. Luego de vectorizar sus cálculos
de las derivadas con respecto a $x^{(i)}$,, debe usar un método similar para vectorizar las derivadas
con respecto a  $θ^{(j)}$ 
</div>


[Haga click acá para regresar a la función `cofiCostFunc` y actualizarla](#cofiCostFunc). 

<font color="red"> No olvide ejecutar de nuevo la celda que contiene la función  `cofiCostFunc` 
de forma que se actualice con la implementación del cómputo del gradiente </font>

In [ ]:
#  Verifique los gradientes corriendo checkcostFunction
utils.checkCostFunction(cofiCostFunc)

<a id="section5"></a>
#### 2.2.3 Función de costo regularizada.

La función de costo para el gradiente colaborativo con regularización es

$$ J(x^{(1)}, \dots, x^{(n_m)}, \theta^{(1)}, \dots, \theta^{(n_u)}) = \frac{1}{2} \sum_{(i,j):r(i,j)=1} \left( \left( \theta^{(j)} \right)^T x^{(i)} - y^{(i,j)} \right)^2 + \left( \frac{\lambda}{2} \sum_{j=1}^{n_u} \sum_{k=1}^{n} \left( \theta_k^{(j)} \right)^2  \right) + \left( \frac{\lambda}{2} \sum_{i=1}^{n_m} \sum_{k=1}^n \left(x_k^{(i)} \right)^2 \right) $$

Debe agregar regularización a los cálculos originales de la función de costo,  $J$. Luego de esto,
la próxima celda corre su función regularizada, y usted debería esperar que la función de costo
produzca  un valor  de 31.34.


[Haga click acá para regresar a  `cofiCostFunc` y actualizarla](#cofiCostFunc)
<font color="red"> No olvide volver a ejecutar la celda que contiene la funci[on de costo `cofiCostFunc`
de forma que se actualice en su implementación de costo regularizada. </font>

In [ ]:
#  Evalúe la función de costo
J, _ = cofiCostFunc(np.concatenate([X.ravel(), Theta.ravel()]),
                    Y, R, num_users, num_movies, num_features, 1.5)
           
print('Cost at loaded parameters (lambda = 1.5): %.2f' % J)
print('              (this value should be about 31.34)')

<a id="section6"></a>
#### 2.2.4 Gradiente Regularizado

Ahora que ha implementado la función de costo regularizada debe proceder a implementar la regularización del
gradiente. Debe agregar a su implementación la función `cofiCostFunc` que retorna el gradiente regularizado
al agregar la contribución de los términos de regularización.   Note que los gradientes de la función 
de costo con regularización están datos por


$$ \frac{\partial J}{\partial x_k^{(i)}} = \sum_{j:r(i,j)=1} \left( \left(\theta^{(j)}\right)^T x^{(i)} - y^{(i,j)} \right) \theta_k^{(j)} + \lambda x_k^{(i)} $$

$$ \frac{\partial J}{\partial \theta_k^{(j)}} = \sum_{i:r(i,j)=1} \left( \left(\theta^{(j)}\right)^T x^{(i)}- y^{(i,j)} \right) x_k^{(i)} + \lambda \theta_k^{(j)} $$


Esto significa que usted debe agregar  $\lambda x^{(i)}$ a la variable  `X_grad[i,:]`  descrita anteriormente,
y agregar $\lambda \theta^{(j)}$ a la variable `Theta_grad[j, :]` descrita anteriormente. 


[Haga click acá para regresar a  `cofiCostFunc` y actualizarla](#cofiCostFunc)
<font color="red"> No olvide volver a ejecutar la celda que contiene la función `cofiCostFunc`  para actualizar
su implementación con gradiente de función regularizada.</font>

Luego de completar el código para calcular los gradientes, la próxima celda corre otro gradiente
para verificar (`utils.checkCostFunction`) que su implementación numérica es correcta.

In [ ]:
#  Verifique los gradientes al correr checkCostFunction
utils.checkCostFunction(cofiCostFunc, 1.5)

### 2.3 Aprendizaje de recomendaciones de películas

Luego de terminar la implementación de las funciones de costo y gradiente de filtros colaborativos, 
debe comenzar el entrenamiento del algoritmo para sugerir recomendaciones para usted mismo. En la
próxima celda, usted puede ingresar sus propia preferencias de forma que, cuando el algoritmo corre, 
pueda obtener recomendaciones!  Hemos llenado algunos valores con nuestras preferencias. Sin embargo
usted debe cambiar esto de acuerdo a sus gustos. La lista de todas las películas y su número en los datos
puede ser encontrada en el archivo `Data/movie_idx.txt`.

In [ ]:
#  Antes de entrenar el modelo de filtros colaborativos primero
#  agregatmos calificaciones que corresponden a un nuevo usuario que
#  acabamos de observar.  Esta parte del código también te permite
#  agregar sus propias calificaciones para los datos de películas.
movieList = utils.loadMovieList()
n_m = len(movieList)

#  inicialice mis calificaciones
my_ratings = np.zeros(n_m)

# Verifique el archivo movie_idx.txt para id de cada película en los datos
# For ejemplo, Toy Story (1995) tiene un ID de 1, y una calificación de 4. 
# Note que el índice es ID-1, dado que comenzamos con un índice 0.
my_ratings[0] = 4

# O suponga que no le gustó Silence of the Lambs (1991), la puedes asignar.
my_ratings[97] = 2

# Seleccionamos unas pocas películas que nos gustan / o no con calificaciones
my_ratings[6] = 3
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63] = 5
my_ratings[65] = 3
my_ratings[68] = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 5

print('New user ratings:')
print('-----------------')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print('Rated %d stars: %s' % (my_ratings[i], movieList[i]))

#### 2.3.1 Recomendaciones

Luego de las que calificaciones adicionales  han sido añadidas al conjunto,
el programa procede a entrenar el modelo de filtros colaborativos. Este aprende
los parámetros X y Theta. Para predecir las calificaciones de la película i 
para el usuario j, necesita calcular (0 (j) ) T x (i). 
La próxima parte del programa calcula las calificaciones para todas las películas
y usuarios y muestra las películas recomendadas (Figura 4), de acuerdo
a las calificaciones que fueron agregadas anteriormente. Note que
usted puede obtener un conjunto diferente de predicciones debido a las
inicializaciones aleatorias.



In [ ]:
#  Ahora entrenamos un modelo de filtros colaborativos en calificación de películas
#  conjunto de  1682 películas y 943 usuarios

#  cargue los datos
data = loadmat(os.path.join('Data', 'ex8_movies.mat'))
Y, R = data['Y'], data['R']

#  Y es una matriz  1682x943 con calificaciones (1-5) de 1682 películas
#  y 943 usuarios

#  R es una matriz 1682x943 matrix, donde R(i,j) = 1 si y solo si el  usuario j califica
#  la película i

#  agrege us propias calificaciones a la matriz de datos
Y = np.hstack([my_ratings[:, None], Y])
R = np.hstack([(my_ratings > 0)[:, None], R])

#  normalice las calificaciones
Ynorm, Ymean = utils.normalizeRatings(Y, R)

#  valores útiles
num_movies, num_users = Y.shape
num_features = 10

# conjunto de parámetros iniciales (Theta, X)
X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

initial_parameters = np.concatenate([X.ravel(), Theta.ravel()])

# Establezca opciones para scipy.optimize.minimize
options = {'maxiter': 100}

# Establezca la regularización
lambda_ = 10
res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,
                                               num_movies, num_features, lambda_),
                        initial_parameters,
                        method='TNC',
                        jac=True,
                        options=options)
theta = res.x

# Despliege la matriz retornada theta en U y W
X = theta[:num_movies*num_features].reshape(num_movies, num_features)
Theta = theta[num_movies*num_features:].reshape(num_users, num_features)

print('Recommender system learning completed.')

Luego de entrenar el modelo puede hacer recomendaciones para calcular la matriz de predicción.

In [ ]:
p = np.dot(X, Theta.T)
my_predictions = p[:, 0] + Ymean

movieList = utils.loadMovieList()

ix = np.argsort(my_predictions)[::-1]

print('Top recommendations for you:')
print('----------------------------')
for i in range(10):
    j = ix[i]
    print('Predicting rating %.1f for movie %s' % (my_predictions[j], movieList[j]))

print('\nOriginal ratings provided:')
print('--------------------------')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print('Rated %d for %s' % (my_ratings[i], movieList[i]))